RoadMap:
Heterogeneous Graph:
  Nodes:
    Actor:
      TotalMovies
      GenreDistribution
    Movie:
      TitleID
      Name
      Runtime
      Genre
      IsAdult
      ReleaseYear
      RuntimeMinutes
      Genres
      TotalRatings
      AvgRating
      Box Office Revenue
  Edges:
    Actor -> Movie (Acted in Movie)
    Actor -> Actor in Shared movies

In [2]:
! pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
! pip install pyg-lib -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
! pip install beautifulsoup4
! pip install mediawikiapi

  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_spline_conv-1.2.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using 

In [3]:
# Perform imports
from csv import reader as CSVReader
from csv import writer as CSVWriter
from operator import itemgetter as itemgetter

In [100]:
# Trim based on minimum start year, "movie" classification and count genre frequencies

inpath = "data/title.basics.tsv"
outpath = "data/title.basics_trimmed.tsv"

minStartYear = 2000
maxStartYear = 2020

genreFreq = {}
with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  i = 0
  j = 0
  for line in reader:
    l = list(line)
    startStr = str(l[5])
    type = str(l[1])
    genres = str(l[8]).split(sep=',')
    if ( (i != 0) and (startStr != "\\N") and (int(startStr) >= minStartYear) and int(startStr) <= maxStartYear and type == "movie"):
      writer.writerow(line)
      for genre in genres:
        if genre != "\\N":
          if genre in genreFreq:
            genreFreq[genre] += 1
          else:
            genreFreq[genre] = 1
      j += 1
    i += 1

print(f"Copied {j} lines out of {i} ({round((j / i) * 100, 2)}%)")

genreFreqItems = genreFreq.items()
print(f"Genres ({len(genreFreqItems)}):")
for genre, freq in sorted(genreFreqItems, key=itemgetter(1), reverse=True):
  print(f"\t{genre}: {freq}")

del genreFreq

Copied 259763 lines out of 10716429 (2.42%)
Genres (26):
	Drama: 90686
	Documentary: 82723
	Comedy: 44836
	Action: 18864
	Thriller: 18573
	Romance: 18374
	Horror: 17214
	Crime: 12699
	Biography: 10592
	Adventure: 9856
	Family: 8743
	Mystery: 8020
	History: 8018
	Music: 7663
	Fantasy: 6249
	Sci-Fi: 5089
	Animation: 4670
	Sport: 4333
	Musical: 2648
	War: 2607
	Adult: 1463
	News: 1317
	Western: 728
	Reality-TV: 373
	Talk-Show: 113
	Game-Show: 10


In [101]:
# Encode genre list for each movie
genreList = list(i[0] for i in genreFreqItems)

inpath = "data/title.basics_trimmed.tsv"
outpath = "data/title.basics_genres_encoded.tsv"

with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  for line in reader:
    l = list(line)
    
    encodedGenres = ""
    genres = str(l[8]).split(sep=',')
    for genre in genreList:
      if genre in genres:
        encodedGenres += '1'
      else:
        encodedGenres += '0' 
    newRow = l[:8]
    newRow.append(encodedGenres)
    writer.writerow(newRow)

In [102]:
# Trim out duplicate references to same movie

inpath = "data/title.basics_genres_encoded.tsv"
outpath = "data/title_unique.tsv"

observedFeatures = set()

numDuplicates = 0

with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  for line in reader:
    l = list(line)
    features = str(l[1:])
    if not features in observedFeatures:
      observedFeatures.add(features)
      writer.writerow(l)
    else:
      numDuplicates += 1    

del observedFeatures
print(f"Removed {numDuplicates} duplicates")

Removed 168 duplicates


In [103]:
# Trim out unnecessary columns (titleType (1), originalTitle (2), isAdult (4), endYear (6))

inpath = "data/title_unique.tsv"
outpath = "data/processed/movies.tsv"

observedFeatures = set()

numDuplicates = 0

with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  for line in reader:
    l = list(line)
    writer.writerow([l[0], l[3], l[5], l[7], l[8]])

In [104]:
# Drop movies that do not contain a runtime length value

inpath = "data/processed/movies.tsv"
outpath = "data/processed/moviesRuntimes.tsv"

numRemoved = 0

with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  for line in reader:
    l = list(line)
    if (l[3] != "\\N"):
      writer.writerow(l)
    else:
      numRemoved += 1

print(f"Removed {numRemoved} entries")

Removed 52939 entries


In [105]:
# Join movies with ratings data, discard those without ratings data or with less than minRatingsCount total ratings

inpathMovies = "data/processed/moviesRuntimes.tsv"
inpathRatings = "data/title.ratings.tsv"
outpath = "data/processed/moviesRatings.tsv"

minRatingsCount = 10000

linesWritten = 0

with open(inpathMovies, "r") as infileMovies, open(inpathRatings, "r") as infileRatings, open(outpath, "w") as outfile:
  readerMovies = CSVReader(infileMovies, delimiter="\t", quotechar=None)
  readerRatings = CSVReader(infileRatings, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  ratingsLine = next(readerRatings)
  ratingsHeader = ratingsLine
  # print(ratingsLine)
  

  for line in readerMovies:
    l = list(line)

    while (ratingsLine[0] < l[0] and len(ratingsHeader) == len(ratingsLine)):
      # print(f"\"{ratingsLine[0]}\"", f"\"{l[0]}\"")
      ratingsLine = next(readerRatings)
    if (ratingsLine[0] == l[0] and int(ratingsLine[2]) >= minRatingsCount):
      l.append(ratingsLine[1])
      l.append(ratingsLine[2])
      writer.writerow(l)
      linesWritten += 1

print(f"Wrote {linesWritten} lines")

Wrote 6560 lines


In [6]:
# Create final movies files
inpath="data/processed/moviesRatings.tsv"

outpathData="data/final/movies.data.tsv"
outpathLabels="data/final/movies.labels.tsv"

existingTitles = set()

with open(inpath, "r") as infile, open(outpathData, "w") as outfileData, open(outpathLabels, "w") as outfileLabels:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writerData = CSVWriter(outfileData, delimiter="\t")
  writerLabels = CSVWriter(outfileLabels, delimiter="\t")
  
  writerLabels.writerow(["TitleID", "TitleName"])
  writerData.writerow(["TitleID", "ReleaseYear", "RuntimeMinutes", "AvgRatings", "NumRatings", "Genres"])

  for line in reader:
    l = list(line)
    # print(l)
    writerLabels.writerow([l[0], l[1]])
    writerData.writerow([l[0], l[2], l[3], l[5], l[6], l[4]])
    existingTitles.add(l[0])

In [7]:
# Trim down "principals" data
inpath="data/title.principals.tsv"
outpath="data/title.principals_trimmed.tsv"

numPrincipals = dict()
categories = set()
moviesPerPerson = dict()

i = 0
with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  first = True
  for line in reader:
    if (first):
      first = False
      continue
    l = list(line)
    # TitleID (0), Ordering (1), NameId (2), Category (3), Job (4), Characters (5)
    if (l[0] in existingTitles):
      if (l[0] in numPrincipals):
        numPrincipals[l[0]] = max(int(l[1]), numPrincipals[l[0]])
      else:
        numPrincipals[l[0]] = (int(l[1]))
      if (l[2] in moviesPerPerson):
        moviesPerPerson[l[2]].add(l[0])
      else:
        moviesPerPerson[l[2]] = set()
        moviesPerPerson[l[2]].add(l[0])
      writer.writerow([l[0], l[2], l[3]])
      categories.add(l[3])
      i += 1

print(f"Rows written: {i}")
print(f"Categories: ({len(categories)}):")
for category in categories:
  print(category)


Rows written: 144422
Categories: (13):
archive_footage
producer
director
cinematographer
actress
casting_director
self
composer
production_designer
archive_sound
editor
actor
writer


In [9]:
# Trim down "names" data
inpath="data/name.basics.tsv"
outpath="data/name.basics_trimmed.tsv"

primaryProfessionsList = set()
i = 0
with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  first = True
  for line in reader:
    if (first):
      first = False
      continue
    l = list(line)
    # NameId (0), PrimaryName (1), BirthYear (2), DeathYear (3), PrimaryProfession (4), KnownForTitles (5)

    for profession in l[4].split(sep=','):
      primaryProfessionsList.add(profession)

    if (l[0] in moviesPerPerson and not l[2] == "\\N"):
      writer.writerow([l[0], l[1], l[2], l[4], len(moviesPerPerson[l[0]])])
      i += 1

print(f"Wrote {i} lines")
print(f"Primary Professions ({len(primaryProfessionsList)}):")
for profession in primaryProfessionsList:
  print(profession)

Wrote 26294 lines
Primary Professions (47):
archive_footage
location_management
costume_designer
editorial_department
special_effects
publicist
music_department
animation_department
assistant_director
legal
transportation_department
assistant
casting_director
actress
make_up_department
accountant
music_artist
talent_agent
stunts
production_designer
editor
\N
production_manager
producer
director
cinematographer
executive
production_department
script_department
art_director
costume_department
miscellaneous
soundtrack
camera_department
podcaster
art_department
set_decorator
visual_effects
sound_department
choreographer
casting_department
composer
archive_sound
manager
electrical_department
actor
writer


In [10]:
# Encode professions data and write labels and values out to final versions
inpath="data/name.basics_trimmed.tsv"
outpathLabels="data/final/person.labels.tsv"
outpathData="data/final/person.data.tsv"

i = 0

with open(inpath, "r") as infile, open(outpathLabels, "w") as outfileLabels, open(outpathData, "w") as outfileData:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writerLabels = CSVWriter(outfileLabels, delimiter="\t")
  writerData = CSVWriter(outfileData, delimiter="\t")

  writerLabels.writerow(["NameID", "PrimaryName"])
  writerData.writerow(["NameID", "BirthYear", "PrimaryProfessions", "MoviesWorkedOn"])

  for line in reader:
    
    l = list(line)
    # NameId (0), PrimaryName (1), BirthYear (2), PrimaryProfession (3), moviesWorkedOn (4)
    encodedProfessions = ""
    professions = str(l[3]).split(sep=',')
    for profession in primaryProfessionsList:
      if profession in professions:
        encodedProfessions += '1'
      else:
        encodedProfessions += '0'


    writerLabels.writerow([l[0], l[1]])
    writerData.writerow([l[0], l[2], encodedProfessions, l[4]])
    i += 1

print(f"Wrote {i} lines")

Wrote 26294 lines


In [11]:
# Establish "WorksInMovie" edges and count shared works between people
inpath="data/title.principals_trimmed.tsv"
outpath="data/final/works_on.edge.tsv"

categoriesList = list(categories)

i = 0
with open(inpath, "r") as infile, open(outpath, "w") as outfile:
  reader = CSVReader(infile, delimiter="\t", quotechar=None)
  writer = CSVWriter(outfile, delimiter="\t")

  writer.writerow(["NameID", "TitleID", "Category"])
  for line in reader:
    l = list(line)
    # TitleId (0), NameID (1), Category (2)
    writer.writerow([l[1], l[0], categoriesList.index(l[2])])

    i += 1

print(f"Rows written: {i}")

Rows written: 144422


In [12]:
# Create "SharesMovie" Edges

# person1 -> person2 -> [movie1, movie2, ..., movieN]

sharedMovies = dict()

for person in moviesPerPerson.keys():
  for movie in moviesPerPerson[person]:
    for person2 in moviesPerPerson.keys():
      if (person < person2): # Should avoid repeats and self edges
        # print(f"P1: {person}, P2: {person2}")
        if (movie in moviesPerPerson[person2]): # Movie is shared by person and person2
          if not (person in sharedMovies):
            sharedMovies[person] = dict()
          if not (person2 in sharedMovies[person]):
            sharedMovies[person][person2] = set()
          sharedMovies[person][person2].add(movie)

In [13]:
# Count elements in shared sets
sharedMoviesCounts = dict()
for person in sharedMovies.keys():
  for person2 in sharedMovies[person].keys():
    if not (person in sharedMoviesCounts):
      sharedMoviesCounts[person] = dict()
    sharedMoviesCounts[person][person2] = len(sharedMovies[person][person2])

In [14]:
# Write calculated edges out to file
outpath="data/final/shares_movies.edge.tsv"

i = 0
with open(outpath, "w") as outfile:
  writer = CSVWriter(outfile, delimiter="\t")

  writer.writerow(["NameID1", "NameID2", "NumShared"])
  for person in sharedMovies.keys():
    for person2 in sharedMovies[person].keys():
      writer.writerow([str(person), str(person2), str(sharedMoviesCounts[person][person2])])
      i += 1

print(f"Rows written: {i}")

Rows written: 1241937
